In [1]:
import torch
import chromadb

from openai import OpenAI

from FlagEmbedding import BGEM3FlagModel
from transformers import XLMRobertaModel, XLMRobertaTokenizer
from sentence_transformers import SentenceTransformer

## Importing models and databases

In [17]:
xlm_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
xlm_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

In [18]:
xlm_client = chromadb.PersistentClient(
    path="/home/murad/Documents/self-study/contextual_embeddings/databases/xlm_collection"
)

xlm_collection = xlm_client.get_collection("pdf_chunks")

In [17]:
bge_model = BGEM3FlagModel(
    model_name_or_path="BAAI/bge-m3",
    use_fp16=True
)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [18]:
bge_client = chromadb.PersistentClient(
    path="/home/murad/Documents/self-study/contextual_embeddings/databases/bge_collection"
)
bge_collection = bge_client.get_collection("pdf_chunks")

In [19]:
labse_model = SentenceTransformer(
    model_name_or_path="sentence-transformers/LaBSE"
)

In [20]:
labse_client = chromadb.PersistentClient(
    path="/home/murad/Documents/self-study/contextual_embeddings/databases/labse_collection"
)
labse_collection = labse_client.get_collection("pdf_chunks")

In [39]:
# Create OpenAI model instance here

In [3]:
openai_client = chromadb.PersistentClient(
    path="/home/murad/Documents/self-study/contextual_embeddings/databases/openai_collection_large"
)
openai_collection = openai_client.get_collection("pdf_chunks")

## Query

In [32]:
query = "Konvolyusional Sinir Şəbəkələri (CNN) nədir və Azərbaycanda hansı sahələrdə istifadə olunur?"

### XLM RoBERTa model result

In [26]:
xlm_model.eval()

with torch.no_grad():
    inputs = xlm_tokenizer(query, return_tensors="pt", padding=True)
    outputs = xlm_model(**inputs)
    query_embedding_xlm = outputs.last_hidden_state[:, 0, :].squeeze(0).numpy().tolist()

In [27]:
xlm_result = xlm_collection.query(
    query_embeddings=query_embedding_xlm,
    n_results=5,
    include=["distances", "metadatas", "documents"]
)

In [28]:
with open("xlm_result.txt", "w", encoding="utf-8") as f:
    f.write(str(xlm_result["documents"]))

### BGE-M3 model result

In [33]:
query_embedding_bge = bge_model.encode(sentences=query,
                                       max_length=1024,
                                       batch_size=12)['dense_vecs']
query_embedding_bge = query_embedding_bge.tolist()

In [34]:
bge_result = bge_collection.query(
    query_embeddings=query_embedding_bge,
    n_results=5,
    include=["distances", "metadatas", "documents"]
)

In [35]:
with open("bge_result.txt", "w", encoding="utf-8") as f:
    f.write(str(bge_result["documents"]))

### LaBSE model result

In [32]:
query_embedding_labse = labse_model.encode(sentences=query)
query_embedding_labse = query_embedding_labse.tolist()

In [33]:
labse_result = labse_collection.query(
    query_embeddings=query_embedding_labse,
    n_results=5,
    include=["distances", "metadatas", "documents"]
)

In [34]:
with open("labse_result.txt", "w", encoding="utf-8") as f:
    f.write(str(labse_result["documents"]))

### OPENAI model result

In [36]:
def get_openai_embeddings(texts, model="text-embedding-3-large"):
    """Generate embeddings for a list of texts using OpenAI API"""
    response = openai_model.embeddings.create(
        input=texts,
        model=model
    )
    
    # Extract embeddings from the response
    embeddings = [item.embedding for item in response.data]
    return embeddings

# Use in your workflow
query_embeddings_openai = get_openai_embeddings(query)

In [37]:
openai_result = openai_collection.query(
    query_embeddings=query_embeddings_openai,
    n_results=5,
    include=["distances", "metadatas", "documents"]
)

In [38]:
openai_result

{'ids': [['chunk_13', 'chunk_8', 'chunk_12', 'chunk_14', 'chunk_7']],
 'distances': [[0.884297152288524,
   0.9313538959731947,
   0.9650503047003142,
   0.9976189089491336,
   1.0531794238621923]],
 'metadatas': [[{'source': 'chunk_13.txt'},
   {'source': 'chunk_8.txt'},
   {'source': 'chunk_12.txt'},
   {'source': 'chunk_14.txt'},
   {'source': 'chunk_7.txt'}]],
 'embeddings': None,
 'documents': [[None, None, None, None, None]],
 'uris': None,
 'data': None}

In [31]:
bge_result

{'ids': [['chunk_5', 'chunk_8', 'chunk_0', 'chunk_7', 'chunk_25']],
 'distances': [[0.8182411739686054,
   0.8713007817483956,
   0.8890924373858589,
   0.9197864670089544,
   0.9218479371924744]],
 'metadatas': [[{'source': 'chunk_5.txt'},
   {'source': 'chunk_8.txt'},
   {'source': 'chunk_0.txt'},
   {'source': 'chunk_7.txt'},
   {'source': 'chunk_25.txt'}]],
 'embeddings': None,
 'documents': [['Maşın Öyrənməsi Alqoritmləri və Modelləri Maşın öyrənməsi alqoritmləri və modelləri, kompüterlərin məlumatlar əsasında öyrənmək və proqnozlar vermək qabiliyyətini təmin edir. Bu alqoritmlər müxtəlif kateqoriyalara bölünür: 4.1. Nəzarətli Öyrənmə (Supervised Learning) Nəzarətli öyrənmədə, alqoritm giriş məlumatları və onlara uyğun çıxış nəticələri əsasında öyrənir. Məqsəd, yeni giriş məlumatları üçün dəqiq çıxış nəticələrini proqnozlaşdıra bilən bir model yaratmaqdır.Əsas nəzarətli öyrənmə alqoritmlərinə aşağıdakılar daxildir: • Xətti Reqressiya (Linear Regression): Bir və ya bir neçə müstəqi